In [7]:
import copy
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import Client, SSHCluster
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

# Macro-Pipeline AHN2 Workflow - Retiling

## Set Run-Specific Input

Choose whether you want to run all input files or run the only input files listed in `filename`.

In [3]:
path_root = pathlib.Path('/data/local/home/eecolidar_webdav/')

# path to LAZ files 
path_input = path_root / '01_Escience/ALS/Netherlands/ahn2/terrain_reclassified'

# path to retiled LAZ files
path_output = path_input.with_name('retiled')

run = 'from_file' # 'all', 'from_file'
filename = 'retile_failed.json'  # if run is 'from_file', set name of file with input file names
assert run in ['all', 'from_file']

In [5]:
laz_files = [el for el in path_input.iterdir() if el.suffix == '.laz']
print('Found: {} LAZ files'.format(len(laz_files)))
if run == 'from_file':
    with open(filename, 'r') as f:
        laz_files_read = json.load(f)
    laz_files_read = [path_input/f for f in laz_files_read]
    # check whether all files are available on dCache
    assert all([f in laz_files for f in laz_files_read]), f'Some of the files in {filename} are not in input dir'
    laz_files = laz_files_read
print('Retrieve and retile: {} LAZ files'.format(len(laz_files)))

Found: 30080 LAZ files
Retrieve and retile: 28435 LAZ files


## Setup Cluster

Setup Dask cluster used for the macro-pipeline calculation.

In [8]:
local_tmp = pathlib.Path('/data/local/tmp')

nprocs_per_node = 2  

# start the cluster
scheduler_node = 'node1'
hosts = [f'node{i}' for i in range(1, 11)]
cluster = SSHCluster(hosts=[scheduler_node] + hosts, 
                     connect_options={'known_hosts': None, 
                                      'username': 'ubuntu', 
                                      'client_keys': '/home/ubuntu/.ssh/id_rsa'},
                     worker_options={'nthreads': 1, 
                                     'nprocs': nprocs_per_node,
                                     'memory_limit': 0,
                                     'local_directory': local_tmp/'dask-worker-space'}, 
                     scheduler_options={'dashboard_address': '8787'})
cluster

distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
distributed.deploy.ssh - INFO - warnings.warn(msg)
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - Clear task state
distributed.deploy.ssh - INFO - distributed.scheduler - INFO -   Scheduler at: tcp://145.100.59.123:8786
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://145.100.59.48:46603'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://145.100.59.48:40547'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         S

SSHCluster('tcp://145.100.59.123:8786', workers=18, threads=18)

## Retiling

The raw point-cloud files are downloaded and retiled to a regular grid.

In [9]:
# details of the retiling schema
grid = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}

In [10]:
# setup input dictionary to configure the retiling pipeline
retiling_input = {
    'setup_local_fs': {'input_folder': path_input.as_posix(),
                       'output_folder': path_output.as_posix()},
    'set_grid': grid,
    'split_and_redistribute': {},
    'validate': {},
}

# write input dictionary to JSON file
with open('retile.json', 'w') as f:
    json.dump(retiling_input, f)

In [ ]:
macro = MacroPipeline()

# add pipeline list to macro-pipeline object and set the corresponding labels
macro.tasks = [Retiler(file.name).config(retiling_input) for file in laz_files]
macro.set_labels([os.path.splitext(file.name)[0] for file in laz_files])

macro.setup_cluster(cluster=cluster)

# run! 
macro.run()

# save outcome results and check that no error occurred before continuing
macro.print_outcome(to_file='retile.out')

failed = macro.get_failed_pipelines()
if failed:
    with open('retile_failed.json', 'w') as f:
        json.dump([pip.label + '.laz' for pip in failed], f)
    raise RuntimeError('Some of the pipelines have failed')

## Terminate cluster

In [13]:
macro.shutdown()

tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://145.100.59.123:8786' processes=20 threads=20>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/usr/local/lib/python3.7/site-packages/distributed/client.py", line 1157, in _heartbeat
    self.scheduler_comm.send({"op": "heartbeat-client"})
  File "/usr/local/lib/python3.7/site-packages/distributed/batched.py", line 117, in send
    raise CommClosedError
distributed.comm.core.CommClosedError
tornado.application - ERROR - Exception in callback <bound method Client._heartbeat of <Client: 'tcp://145.100.59.123:8786' processes=20 threads=20>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/tornado/ioloop.py", line 907, in _run
    return self.callback()
  File "/usr/local/lib/python3.7/site-packages/distributed/client.py", line 1157, in _heartbeat
   